In [12]:
import sympy as sy
from sympy import sin, cos, pi, sqrt
import matplotlib.pyplot as plt

In [13]:
def HTM(theta, a, d, alpha):
    return sy.Matrix([
        [cos(theta), -sin(theta), 0, a],
        [sin(theta)*cos(alpha), cos(theta)*cos(alpha), -sin(alpha), -d*sin(alpha)],
        [sin(theta)*sin(alpha), cos(theta)*sin(alpha), cos(alpha), d*cos(alpha)],
        [0, 0, 0, 1],
    ])

l0, l1, l2, r9, r1, r2 = sy.symbols("l0, l1, l2, r9, r1, r2")
q = sy.Matrix(sy.MatrixSymbol('q', 3, 1))
dq = sy.Matrix(sy.MatrixSymbol('dq', 3, 1))


T0 = HTM(0, 0, 0, 0)
T1 = HTM(q[0,0]+pi/2, 0, l0, 0)
T2 = HTM(q[1,0], 0, 0, -pi/2)
T3 = HTM(q[2,0], l1, 0, 0)
T4 = HTM(0, l2, 0, 0)

Ts = [T0, T1, T2, T3, T4]
T_abs = [T0]
for T in Ts[1:]:
    T_abs.append(T_abs[-1] * T)


In [14]:
T_abs[0][:3, 3]

Matrix([
[0],
[0],
[0]])

In [15]:
T_abs[1][:3, 3]

Matrix([
[ 0],
[ 0],
[l0]])

In [16]:
T_abs[2][:3, 3]

Matrix([
[ 0],
[ 0],
[l0]])

In [17]:
T_abs[3][:3, 3]

Matrix([
[-l1*sin(q[0, 0])*cos(q[1, 0])],
[ l1*cos(q[0, 0])*cos(q[1, 0])],
[         l0 - l1*sin(q[1, 0])]])

In [18]:
T_abs[4][:3, 3]

Matrix([
[-l1*sin(q[0, 0])*cos(q[1, 0]) + l2*(sin(q[0, 0])*sin(q[1, 0])*sin(q[2, 0]) - sin(q[0, 0])*cos(q[1, 0])*cos(q[2, 0]))],
[l1*cos(q[0, 0])*cos(q[1, 0]) + l2*(-sin(q[1, 0])*sin(q[2, 0])*cos(q[0, 0]) + cos(q[0, 0])*cos(q[1, 0])*cos(q[2, 0]))],
[                                  l0 - l1*sin(q[1, 0]) + l2*(-sin(q[1, 0])*cos(q[2, 0]) - sin(q[2, 0])*cos(q[1, 0]))]])

In [19]:
ee = sy.Matrix([[l2, 0, 0, 1]]).T
ee_ = (T_abs[3] * ee)[:3, :]
sy.simplify(ee_)

Matrix([
[(-l1*cos(q[1, 0]) - l2*cos(q[1, 0] + q[2, 0]))*sin(q[0, 0])],
[ (l1*cos(q[1, 0]) + l2*cos(q[1, 0] + q[2, 0]))*cos(q[0, 0])],
[           l0 - l1*sin(q[1, 0]) - l2*sin(q[1, 0] + q[2, 0])]])

In [20]:
ori = sy.Matrix([
    [-sin(q[0,0])*(l1*cos(q[1,0]) + l2*cos(q[1,0] + q[2,0]))],
    [cos(q[0,0])*(l1*cos(q[1,0]) + l2*cos(q[1,0] + q[2,0]))],
    [l0 - l1*sin(q[1,0]) - l2*sin(q[1,0] + q[2,0])]
])
ori

Matrix([
[-(l1*cos(q[1, 0]) + l2*cos(q[1, 0] + q[2, 0]))*sin(q[0, 0])],
[ (l1*cos(q[1, 0]) + l2*cos(q[1, 0] + q[2, 0]))*cos(q[0, 0])],
[           l0 - l1*sin(q[1, 0]) - l2*sin(q[1, 0] + q[2, 0])]])

In [21]:
sy.simplify(ori - T_abs[4][:3, 3])

Matrix([
[0],
[0],
[0]])

In [22]:

os = [T[0:3, 3:4] for T in T_abs]
Rxs = [T[0:3, 0:1] for T in T_abs]
Rys = [T[0:3, 1:2] for T in T_abs]
Rzs = [T[0:3, 2:3] for T in T_abs]

Jos = [o.jacobian(q) for o in os]
JRxs = [r.jacobian(q) for r in Rxs]
JRys = [r.jacobian(q) for r in Rys]
JRzs = [r.jacobian(q) for r in Rzs]

t = sy.Symbol("t")
q1 = sy.Function("q1")
q2 = sy.Function("q2")
q3 = sy.Function("q3")


T_abs_ = []
for T in T_abs:
    T_ = T.subs([
        (q[0,0], q1(t)),
        (q[1,0], q2(t)),
        (q[2,0], q3(t)),
    ])
    T_abs_.append(T_)


os_ = [T[0:3, 3:4] for T in T_abs_]
Rxs_ = [T[0:3, 0:1] for T in T_abs_]
Rys_ = [T[0:3, 1:2] for T in T_abs_]
Rzs_ = [T[0:3, 2:3] for T in T_abs_]

q_ = sy.Matrix([
    [q1(t)],
    [q2(t)],
    [q3(t)],
])
Jos_ = [o.jacobian(q_) for o in os_]
JRxs_ = [r.jacobian(q_) for r in Rxs_]
JRys_ = [r.jacobian(q_) for r in Rys_]
JRzs_ = [r.jacobian(q_) for r in Rzs_]

Jos_dot_ = [sy.diff(J, t) for J in Jos_]
JRxs_dot_ = [sy.diff(J, t) for J in JRxs_]
JRys_dot_ = [sy.diff(J, t) for J in JRys_]
JRzs_dot_ = [sy.diff(J, t) for J in JRzs_]


Jos_dot = []
JRxs_dot = []
JRys_dot = []
JRzs_dot = []
for Js, newJs in zip((Jos_dot_, JRxs_dot_, JRys_dot_, JRzs_dot_), (Jos_dot, JRxs_dot, JRys_dot, JRzs_dot)):
    for J in Js:
        newJs.append(J.subs([
        (sy.Derivative(q1(t),t), dq[0, 0]),
        (sy.Derivative(q2(t),t), dq[1, 0]),
        (sy.Derivative(q3(t),t), dq[2, 0]),
        (q1(t), q[0, 0]),
        (q2(t), q[1, 0]),
        (q3(t), q[2, 0]),
    ]))


os = [sy.expand(e) for e in os]
Rxs = [sy.expand(e) for e in Rxs]
Rys = [sy.expand(e) for e in Rys]
Rzs = [sy.expand(e) for e in Rzs]
Jos = [sy.expand(e) for e in Jos]
JRxs = [sy.expand(e) for e in JRxs]
JRys = [sy.expand(e) for e in JRys]
JRzs = [sy.expand(e) for e in JRzs]
Jos_dot = [sy.expand(e) for e in Jos_dot]
JRxs_dot = [sy.expand(e) for e in JRxs_dot]
JRys_dot = [sy.expand(e) for e in JRys_dot]
JRzs_dot = [sy.expand(e) for e in JRzs_dot]

expr_all = [os, Rxs, Rys, Rzs, Jos, JRxs, JRys, JRzs, Jos_dot, JRxs_dot, JRys_dot, JRzs_dot]
names = [str(i) for i in range(5)]
expr_name = [
    ["o_" + n for n in names],
    ["rx_" + n for n in names],
    ["ry_" + n for n in names],
    ["rz_" + n for n in names],
    ["jo_" + n for n in names],
    ["jrx_" + n for n in names],
    ["jry_" + n for n in names],
    ["jrz_" + n for n in names],
    ["jo_" + n + "_dot" for n in names],
    ["jrx_" + n + "_dot" for n in names],
    ["jry_" + n + "_dot" for n in names],
    ["jrz_" + n + "_dot" for n in names],
]

***

In [23]:
Ix1, Ix2, Ix3, Iy1, Iy2, Iy3, Iz1, Iz2, Iz3 = sy.symbols("Ix1, Ix2, Ix3, Iy1, Iy2, Iy3, Iz1, Iz2, Iz3")
m1, m2, m3 = sy.symbols("m1, m2, m3")
M = sy.Matrix(sy.MatrixSymbol('M', 3, 3))
th1 = q[0,0]
th2 = q[1,0]
th3 = q[2,0]

M[0,0] = Iy2*sin(th2)**2 + Iy3*sin(th2+th3)**2 + Iz1 + Iz2*cos(th2)**2 \
    + Iz3*cos(th2+th3)**2 + m2*r1**2*cos(th2)**2 \
        + m3*(l1*cos(th2) + r2*cos(th2+th3))**2
M[0,1] = 0
M[0,2] = 0

M[1,0] = 0
M[1,1] = Ix2 + Ix3 + m3*l1**2 + m2*r1**2 + m3*r2**2 + 2*m3*l1*r2*cos(th3)
M[1,2] = Ix3 + m3*r2**2 + m3*l1*r2+cos(th3)

M[2,0] = 0
M[2,1] = Ix3 + m3*r2**2 + m3*l1*r2*cos(th3)
M[2,2] = Ix3 + m3*r2**2

M

Matrix([
[Iy2*sin(q[1, 0])**2 + Iy3*sin(q[1, 0] + q[2, 0])**2 + Iz1 + Iz2*cos(q[1, 0])**2 + Iz3*cos(q[1, 0] + q[2, 0])**2 + m2*r1**2*cos(q[1, 0])**2 + m3*(l1*cos(q[1, 0]) + r2*cos(q[1, 0] + q[2, 0]))**2,                                                                    0,                                        0],
[                                                                                                                                                                                               0, Ix2 + Ix3 + l1**2*m3 + 2*l1*m3*r2*cos(q[2, 0]) + m2*r1**2 + m3*r2**2, Ix3 + l1*m3*r2 + m3*r2**2 + cos(q[2, 0])],
[                                                                                                                                                                                               0,                               Ix3 + l1*m3*r2*cos(q[2, 0]) + m3*r2**2,                           Ix3 + m3*r2**2]])

In [24]:
g = sy.Symbol("g")
G = sy.Matrix(sy.MatrixSymbol('M', 3, 1))

G[0,0] = 0
G[1,0] = -(m2*g*r1 + m3*g*l1)*cos(th2) - m3*g*r2*cos(th2+th3)
G[2,0] = -m3*g*r2*cos(th2+th3)

G

Matrix([
[                                                                  0],
[-g*m3*r2*cos(q[1, 0] + q[2, 0]) + (-g*l1*m3 - g*m2*r1)*cos(q[1, 0])],
[                                    -g*m3*r2*cos(q[1, 0] + q[2, 0])]])

In [25]:
Gamma = [
    [
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
    ],
    [
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
    ],
    [
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
    ],
]

Gamma[0][0][1] = (Iy2 - Iz2 - m2*r1**2)*cos(th2)*sin(th2) \
    + (Iy3 - Iz3)*cos(th2+th3)*sin(th2+th3)\
        -m3*(l1*cos(th2) + r2*cos(th2+th3))*(l1*sin(th2) + r2*sin(th2+th3))

Gamma[0][0][2] = (Iy3 - Iz3)*cos(th2+th3)*sin(th2+th3) \
    -m3*r2*(l1*cos(th2) + r2+cos(th2+th3))*sin(th2+th3)

Gamma[0][1][0] = (Iy2 - Iz2 - m2*r1**2)*cos(th2)*sin(th2) \
    + (Iy3 - Iz3)*cos(th2+th3)*sin(th2+th3) \
        -m3*(l1*cos(th2) + r2*cos(th2+th3)) * (l1*sin(th2) + r2*sin(th2+th3))

Gamma[0][2][0] = (Iy3 - Iz3)*cos(th2+th3)*sin(th2+th3) \
    - m3*r2*(l1*cos(th2) + r2+cos(th2+th3)) * sin(th2+th3)

Gamma[1][0][0] = -(Iy2 - Iz2 - m2*r1**2)*cos(th2)*sin(th2) \
    - (Iy3 - Iz3)*cos(th2+th3)*sin(th2+th3) \
        + m3*(l1*cos(th2) + r2*cos(th2+th3)) * (l1*sin(th2) + r2+sin(th2+th3))

Gamma[1][1][2] = -l1*m3*r2*sin(th3)
Gamma[1][2][1] = Gamma[1][1][2]
Gamma[1][2][2] = Gamma[1][1][2]

Gamma[2][0][0] = -(Iy3 - Iz3)*cos(th2+th3)*sin(th2+th3) \
    + m3*r2*(l1*cos(th2) + r2*cos(th2+th3)) * sin(th2+th3)

Gamma[2][1][1] = l1*m3*r2*sin(th3)


C = sy.Matrix(sy.MatrixSymbol("C", 3, 3))
for i in range(3):
    for j in range(3):
        tmp = 0
        for k in range(3):
            tmp += Gamma[i][j][k] * dq[k]
        C[i, j] = tmp

C

Matrix([
[(-m3*r2*(l1*cos(q[1, 0]) + r2 + cos(q[1, 0] + q[2, 0]))*sin(q[1, 0] + q[2, 0]) + (Iy3 - Iz3)*sin(q[1, 0] + q[2, 0])*cos(q[1, 0] + q[2, 0]))*dq[2, 0] + (-m3*(l1*sin(q[1, 0]) + r2*sin(q[1, 0] + q[2, 0]))*(l1*cos(q[1, 0]) + r2*cos(q[1, 0] + q[2, 0])) + (Iy3 - Iz3)*sin(q[1, 0] + q[2, 0])*cos(q[1, 0] + q[2, 0]) + (Iy2 - Iz2 - m2*r1**2)*sin(q[1, 0])*cos(q[1, 0]))*dq[1, 0], (-m3*(l1*sin(q[1, 0]) + r2*sin(q[1, 0] + q[2, 0]))*(l1*cos(q[1, 0]) + r2*cos(q[1, 0] + q[2, 0])) + (Iy3 - Iz3)*sin(q[1, 0] + q[2, 0])*cos(q[1, 0] + q[2, 0]) + (Iy2 - Iz2 - m2*r1**2)*sin(q[1, 0])*cos(q[1, 0]))*dq[0, 0], (-m3*r2*(l1*cos(q[1, 0]) + r2 + cos(q[1, 0] + q[2, 0]))*sin(q[1, 0] + q[2, 0]) + (Iy3 - Iz3)*sin(q[1, 0] + q[2, 0])*cos(q[1, 0] + q[2, 0]))*dq[0, 0]],
[                                                                                                                                                     (m3*(l1*cos(q[1, 0]) + r2*cos(q[1, 0] + q[2, 0]))*(l1*sin(q[1, 0]) + r2 + sin(q[1, 0] + q[2, 0])) - 

In [26]:
from sympy.printing.numpy import NumPyPrinter

names = [str(i) for i in range(5)]

common_w = "import numpy as np\nfrom math import cos as c\nfrom math import sin as s\nfrom math import tan as ta\nfrom math import sqrt as sq\n"

#numba_word_q = "@njit(\"f8[:, :](f8[:, :])\")\n"
#numba_word_q_dq = "@njit(\"f8[:, :](f8[:, :], f8[:, :])\")\n"

with open("src_py_/htm.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, os):
        numpy_word = "def o_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

    for name, z in zip(names, Rxs):
        numpy_word = "def rx_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

    for name, z in zip(names, Rys):
        numpy_word = "def ry_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

    for name, z in zip(names, Rzs):
        numpy_word = "def rz_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")


with open("src_py_/Jos.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, Jos):
        numpy_word = "def jo_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

with open("src_py_/JRxs.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, JRxs):
        numpy_word = "def jrx_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

with open("src_py_/JRys.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, JRys):
        numpy_word = "def jry_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

with open("src_py_/JRzs.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, JRzs):
        numpy_word = "def jrz_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")


with open("src_py_/Jo_dots.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, Jos_dot):
        numpy_word = "def jo_" + name + "_dot(q, dq):\n    return "
        #f.write(numba_word_q_dq)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

with open("src_py_/JRx_dots.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, JRxs_dot):
        numpy_word = "def jrx_" + name + "_dot(q, dq):\n    return "
        #f.write(numba_word_q_dq)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

with open("src_py_/JRy_dots.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, JRys):
        numpy_word = "def jry_" + name + "_dot(q, dq):\n    return "
        #f.write(numba_word_q_dq)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

with open("src_py_/JRz_dots.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, JRzs):
        numpy_word = "def jrz_" + name + "_dot(q, dq):\n    return "
        #f.write(numba_word_q_dq)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")


with open("src_py_/dynamics.py", "w") as f:
    f.write(common_w)
    numpy_word = "def M(q):\n    return "
    #f.write(numba_word_q)
    f.write(numpy_word)
    f.write(NumPyPrinter().doprint(M))
    f.write("\n")
    
    numpy_word = "def C(q, dq):\n    return "
    #f.write(numba_word_q)
    f.write(numpy_word)
    f.write(NumPyPrinter().doprint(C))
    f.write("\n")
    
    numpy_word = "def G(q):\n    return "
    #f.write(numba_word_q)
    f.write(numpy_word)
    f.write(NumPyPrinter().doprint(G))
    f.write("\n")


In [27]:
def translate_hoge(original, done):
    with open(original, "r") as f, open(done, "w") as g:
        file_data = f.readlines()
        for line in file_data:
            line = line.replace('numpy', 'np').replace('1/2', '0.5').replace('(0.5)', '0.5')
            line = line.replace('np.cos', 'c').replace('np.sin', 's').replace('np.sqrt', 'sq')
            #line = line.replace('(q)', '(q, L1, L3, L4, L5_1, L5_2, L7, L8)')
            #line = line.replace('(q, dq)', '(q, dq, L1, L3, L4, L5_1, L5_2, L7, L8)')
            # line = line.replace(']])', ']], dtype=np.float64)')
            # line = line.replace('[0, 0, 0, 0, 0, 0, 0]', '[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]')
            # line = line.replace('[0]', '[0.0]').replace(' 0]],', ' 0.0]],').replace('[1]', '[1.0]').replace('[[0,', '[[0.0,').replace('0.0, 0],', '0.0, 0.0],')
            line = line.replace('import np as np', 'import numpy as np')
            g.write(line)


done_name = "done"

translate_hoge("src_py_/htm.py", done_name+"/htm.py")
translate_hoge("src_py_/Jos.py", done_name+"/Jos.py")
translate_hoge("src_py_/JRxs.py", done_name+"/JRxs.py")
translate_hoge("src_py_/JRys.py", done_name+"/JRys.py")
translate_hoge("src_py_/JRzs.py", done_name+"/JRzs.py")
translate_hoge("src_py_/Jo_dots.py", done_name+"/Jo_dots.py")
translate_hoge("src_py_/JRx_dots.py", done_name+"/JRx_dots.py")
translate_hoge("src_py_/JRy_dots.py", done_name+"/JRy_dots.py")
translate_hoge("src_py_/JRz_dots.py", done_name+"/JRz_dots.py")
translate_hoge("src_py_/dynamics.py", done_name+"/dynamics.py")